# Analisi lessicale

Il primo e fondamentale ruolo dell'analisi lessicale è la tokenizzazione del codice sorgente di input. Ogni token è un elemento linguistico per la successiva fase di analisi sintattica.

Lo scanner, colui che svolge l'analisi lessicale, esegue anche altre importanti attività:

- Riconoscere e filtrare commenti, spazi e altri caratteri di separazione
- Registrare la posizione di occorrenza nel file di input dei vari token. Questo permette in un secondo momento, qualora dovessero riscontrarsi errori, di emettere messaggi di diagnostica precisi
- Procedere all'eventuale espansione delle macro

Noi analizzeremo principalmente il procedimento di tokenizzazione.

## Espressioni regolari pure

Una espressione regolare è una scrittura che rappresenta un linguaggio di un dato alfabeto. La definizione di espressioni regolari pure procede per ricorrenza partendo proprio dall'alfabeto di riferimento.

Le due seguenti definizioni costituiscono la base della costruzione ricorsiva:

- Se $E$ e $F$ sono due espressioni regolari che denotano rispettivamente i linguaggi $L_E$ e $L_F$, allora la scrittura $EF$ è a sua volta una espressione regolare che denota il seguente linguaggio $$L_{EF} = \left \{ z : z = xy, x \in L_E, y \in L_F \right \}$$

- Se $E$ ed $F$ sono espressioni regolari che denotano rispettivamente i linguaggi $L_E$ e $L_F$, allora la scrittura $E|F$ è anch'essa un'espressione regolare che denota il seguente linguaggio $$L_{E|F} = \left \{ z : z \in L_E \quad oppure \quad z \in L_F \right \}$$
Il linguaggio dell'unione di due espressioni regolari $L_{E|F}$ è possibile scriverlo anche come $L_E|L_F$.

Vediamo un primo esempio per comprendere il significato.  
Consideriamo il seguente alfabeto $\left \{ 0, 1 \right \}$:

- $\textbf{00}$ è la concatenazione di due espressioni regolari uguali, ovvero $\textbf{0}$, quindi il linguaggio corrispondente risulta $\left \{ 00 \right \}$
- $\textbf{0|1}$ è l'unione di due espressioni regolari diverse, ovvero $\textbf{0}$ e $\textbf{1}$, quindi il linguaggio corrispondente risulta $\left \{ 0, 1 \right \}$

Anche gli operatori nelle espressioni regolari hanno ordini di precedenza. L'operatore di concatenazione ha la precedenza sull'operatore di unione. Inoltre, come nell'analisi matematica, è possibile utilizzare le parentesi tonde () per forzare, cambiare, l'ordine di precedenza.

Facciamo un esempio:

- La seguente espressione regolare $\textbf{00}|\textbf{1}$ può essere interpretato come $(\textbf{00})|\textbf{1}$ e risulta il seguente linguaggio $\left \{ 00, 1 \right \}$
- La seguente espressione regolare $\textbf{0}(\textbf{0}|\textbf{1})$ ha invece il seguente linguaggio $\left \{ 00, 01 \right \}$

Completiamo la definizione con le seguenti proprietà:

- Se $L$ è un linguaggio, la potenza n-esima di $L$ si definisce come la concatenazione tra $L$ e $L^{n-1}$
$$L^n = LL^{n-1} = L^{n-1}L, n>0$$
- La chiusura riflettiva e transitiva di un linguaggio $L$ si definisce nel modo seguente $$L^* = \bigcup_{n=0}^{\infty} L^n$$
- Se $E$ è un'espressione regolare che denota il linguaggio $L_E$, allora $E^*$ è un'espressione regolare che denota il linguaggio $L_E^*$

Vediamo un esempio per ognuna delle ultime propietà viste.

- Se $L = \left \{ 1, 10 \right \}$, avremo che $$L^2 = \left \{ 11, 110, 101, 1010 \right \}$$ $$L^3 = \left \{ 111, 1110, 1101, 11010, 1011, 10110, 10101, 101010 \right \}$$
- Se $L = \left \{ 1, 10 \right \}$, avremo che $$L^* = \left \{ \varepsilon, L^1, L^2, L^3, ... \right \}$$ $$L^* = \left \{ \varepsilon, 1, 10, 11, 110, 101, 1010, 111, 1110, 1101, 11010, 1011, 10110, 10101, 101010, ... \right \}$$

Con $\varepsilon$ intendiamo una stringa nulla.

## Metacaratteri

Definiamo metacaratteri quei simboli che non fanno parte del linguaggio ma che servono per specificare formalmente il linguaggio. Nel caso delle espressioni regolari su un alfabeto $A$, sono metacaratteri i simboli usati per le espressioni regolati. Sono metacaratteri anche i seguenti simboli $*$, $|$.

Nel caso uno dei metacaratteri fosse anche un simbolo del linguaggio stesso, allora quest'ultimo deve essere preceduto da un ```\```. Questo carattere viene chiamato ```escaped```.

Python possiede molti metacaratteri, alcuni esempi sono i seguenti:

In [1]:
# \n metacarattere new line
string = "Ciao\nMondo!"
print(string)

Ciao
Mondo!


In [2]:
# \t metacarattere tabulazione
string = "Ciao\tMondo!"
print(string)

Ciao	Mondo!


Python quando incontra un "\" non lo interpreta come carattere della stirnga, ma bensì come un segnale che ciò che dovrà essere interpretata è una sequenza di caratteri.

In python esistono le cosidette ```raw string```, ovvero stringhe in cui ogni carattere è interpretato singolarmente.

In [4]:
# raw string
string = r"Ciao\nMondo!"
print(string)

Ciao\nMondo!


Le raw string giocano un ruolo importante nella specifica delle espressioni regolari.

## Espressioni regolari estese

Utilizzando solamente le proprietà delle espressioni regolari esposte precedentemente, ovvero per le espressioni regolari pure, si rinscontrerebbero limitazioni inaccettabili in applicazioni reali vere. Come ad esempio il volere costruire un linguaggio, usando le espressioni regolari, costituito dall'instero alfabeto ASCII a differenza del carattere A che non lo vogliamo. L'unico modo consiste nello scrivere una unione di 255 espressioni regolari, dove la 256-esima è il carattere A.

Notiamo che l'utilizzo delle espressioni regolari pure è molto stringente. Detto ciò, si adottano delle ulteriori regole e convenzioni per rendere più agile l'uso delle espressioni regolari. Vediamole:

- $E^+$ denota l'espressione regolare £EE^*£ $$0^+ = 00^* = \left \{ 0, 00, 000, ...\right \}$$
- $E^n$ = $EE...E \ n \ volte$
- $E^n_m = E^M|E^{m+1}|...|E^n$ denota un intervallo di potenze
- $[c_1c_2...c_n]$ denota l'insieme di caratteri $\left \{ c_1, c_2, ..., c_n \right \}$
- $[\hat{} c_1c_2...c_n]$ denota l'insieme di caratteri dell'alfabeto esclusi $c_1, c_2, ..., c_n$
- $.$ denota un qualsiasi carattere dell'alfabeto di riferimento

Nella maggior parte di strumenti che utilizzano le espressioni regolari non si possono scrivere apici o pendici, di conseguenza si utilizzano le seguenti scritture:

- $E\left \{ n \right \}$ per $E^n$
- $E\left \{ m, n \right \}$ per $E^n_m$
- $E\left \{ ,n \right \}$ per $E^n_0$
- $E\left \{ m, \right \}$ per $E^{\infty}_m$

## Espressioni regolari in python

Per utilizzare le espressioni regolari in python si utilizza il modulo ```re```.

In [1]:
import re

Il module re contiene diverse funzioni. La prima che vediamo è ```re.compile```. Questa funzione riceve in input una stringa che rappresenta la nostra espressione regolare e restituisce in output un oggetto di tipo pattern.

L'oggetto pattern restituito ci permette di utilizzare diversi metodi di ricerca su un testo che deve essere specificato. Il primo metodo che utilizziamo è ```match```. Questo metodo riceve come parametro una stringa di testo sul quale applicare l'espressione regolare. Match restituisce un oggetto di tipo match nel caso di risultato positivo altrimenti restituisce None nel caso negativo. Il metodo match controlla se la stringa di testo da valutare incomicia con l'espressione regolare.

In [4]:
pattern = re.compile("In")

In [8]:
print(pattern.match("Informatica"))

<re.Match object; span=(0, 2), match='In'>


In [6]:
print(pattern.match("informatica"))

None


Sull'oggetto di tipo match è possibile utilizzare tre metodi:

- start(), inizio del matching dell'espressione regolare
- end(), fine del matching dell'espressione regolare
- group(), insieme dei caratteri che formano il match

In [10]:
match = pattern.match("Informatica")

print("Match:", match)

print("Start:\t", match.start())
print("End:\t", match.end())
print("Group:\t", match.group())

Match: <re.Match object; span=(0, 2), match='In'>
Start:	 0
End:	 2
Group:	 In


Un secondo metodo che possiamo utilizzare su un oggetto di tipo pattern è ```search```. È analogo al metodo match con la differenza che search cerca una corrispondenza in tutta la stringa di testo e non solo all'inizio di essa come il match. In caso di corrispondenza restituisce un oggetto match altrimenti None.

In [12]:
match = pattern.match("Ingegneria Informatica")
search = pattern.search("Ingegneria Informatica")

print("Match:", match)

print("Start:\t", match.start())
print("End:\t", match.end())
print("Group:\t", match.group())

print("Search:", search)

print("Start:\t", search.start())
print("End:\t", search.end())
print("Group:\t", search.group())

Match: <re.Match object; span=(0, 2), match='In'>
Start:	 0
End:	 2
Group:	 In
Search: <re.Match object; span=(0, 2), match='In'>
Start:	 0
End:	 2
Group:	 In


In [14]:
match = pattern.match("ingegneria Informatica")
search = pattern.search("ingegneria Informatica")

print("Match:", match)

if match:
    print("Start:\t", match.start())
    print("End:\t", match.end())
    print("Group:\t", match.group())

print("Search:", search)

print("Start:\t", search.start())
print("End:\t", search.end())
print("Group:\t", search.group())

Match: None
Search: <re.Match object; span=(11, 13), match='In'>
Start:	 11
End:	 13
Group:	 In


Come si nota dai due esempi sopra, il metodo search cerca si in tutta la stringa di testo una corrispondenza, ma non trova tutte le corrispondenze. Infatti appena trovata una, esso termina. Nel caso volessimo trovare tutte le corrispondenze nel testo, il metodo da utilizzare è un altro.

Il metodo in questione ```findall```, è il terzo metodo che analizziamo. Esso, come accennato, è quel metodo che ci permette di trovare tutte le corrispondenze in tutta la stringa di testo.

In [15]:
findall = pattern.findall("Ingegneria Informatica")

print("Findall:", findall)

Findall: ['In', 'In']


Findall ci permette sostanzialmente di effettuare molteplici search finchè non termina il testo. Inoltre, findall ci ritorna direttamente la corrispondenza e non un oggetto match.

Finall è possibile implementarlo semplicemente con un search ed un while.

In [16]:
index = 0
while match := pattern.search("Ingegneria Informatica", index):
    print(f"{match.group()} at {match.start()}")
    index = match.end()

In at 0
In at 11


L'ultimo metodo importante che utilizziamo su di un oggetto pattern è ```finditer```. Finditer è analoogo a findall con la sola differenza che restituisce un iterabile anzichè una lista.

In [18]:
finditer = pattern.finditer("Ingegneria Informatica")

print("Finditer:", finditer)

for match in finditer:
    print(match)

Finditer: <callable_iterator object at 0x114464ca0>
<re.Match object; span=(0, 2), match='In'>
<re.Match object; span=(11, 13), match='In'>


Solitamente, nella pratica, non si usa mai compilare il pattern separatamente, ma piuttosto specificare l'espressione regolare ed il testo nello stesso metodo.

Proviamo a concludere le espressioni regolari con un esempio un pò più interessante.

Vogliamo trovare tutti i match dell'espressione regolare ```[CG]{3, }``` in una stringa che rappresenta una porzione di DNA umano.

La stringa in input è la seguente:

> "GAGGATTAGGTCTGGGACACGGAGAGGGTGTCCCTTCCTCATCCCCAGGT"

I risultati che attendiamo sono i seguenti:

- GGG alla posizione 13
- CGG alla posizione 19
- GGG alla posizione 25
- CCC alla posizione 31
- CCCC alla posizione 42

In [38]:
dna = "GAGGATTAGGTCTGGGACACGGAGAGGGTGTCCCTTCCTCATCCCCAGGT"

matches = re.finditer(r"[CG]{3,}", dna)

for match in matches:
    print(f"{match.group()} at {match.start()}")

GGG at 13
CGG at 19
GGG at 25
CCC at 31
CCCC at 42


Una cosa molto importante quando si specificano le espressioni regolari è il come le si scrivono. Basta aggiungere uno spazio dopo la virgola dell'espressione regolare che il risultato diventa None.

In [39]:
dna = "GAGGATTAGGTCTGGGACACGGAGAGGGTGTCCCTTCCTCATCCCCAGGT"

matches = re.finditer(r"[CG]{3, }", dna)

for match in matches:
    print(f"{match.group()} at {match.start()}")

## Generatori di scanner

Un generatore di scanner è un software che viene istruito sulle espressioni regolari da riconoscere che restituisce un programma in grado di riconoscere effettivamente i token in una stringa di input.

Il più famoso stumento di generatore di scanner è ```Lex```, il quale produce in output un codice C compatibile. In python il generatore più comune è ```PLY```.  
Lex lo guarderemo velocemente, mentre PLY lo utilizzeremo più intensamente.

La struttura logica di Lex si presenta in questo modo.

![Image 5](images/image-5.png)

Un generico programma Lex contiene tre sezioni separate da una riga contenente due soli ```%%```.

```
Dichiarazioni
%%
Regole di traduzione
%%
Funzioni ausiliarie
```

- Dichiarazioni

    La sezione dichiarazioni contiene definizioni di costanti e variabili. In più, è la sezione nel quale si possono dare nomi alle espressioni regolari da potere utilizzare nelle sezioni successive.

- Reegole di traduzione

    La sezione più importante, dove viene definita la logica del programma. Questa sezione contiene le descrizioni dei token da riconoscere e le corrispondenti azioni che devono essere eseguite dallo scanner.

- Funzioni ausiliarie

    Questa sezione permette di definire funzioni ausiliarie nel caso di bisogno. Nel caso in cui lo scanner non è utilizzato come routine del parser o di altri programmi, la sezione funzioni ausiliarie può contenere la funzione main del programma. In questo caso, all'interno del main dovrà essere presente la chiamata allo scanner.

Il seguente codice è il programma equivalente al comando ```wc``` di UNIX.


```l
%{
    unsigned charCount = 0;
    unsigned wordCount = 0;
    unsigned lineCount = 0;
%}

%option noyywrap

word [^ \t\n]+
eol \n

%%

{word}  {
    wordCount++;
    charCount += yyleng;
}

{eol}   {
    charCount++;
    lineCount++;
}

.       {
    charCount++;
}

%%

int main()
{
    yylex();
    printf("%i %i %i\n", lineCount, wordCount, charCount);
}
```

Proviamo ad utilizzarlo.

In [40]:
ls -lah

total 80
drwxr-xr-x@  7 luigimalaguti  staff   224B Dec 10 18:07 ./
drwxr-xr-x@ 10 luigimalaguti  staff   320B Dec 10 18:07 ../
-rw-r--r--   1 luigimalaguti  staff    11K Dec  8 18:00 1_Compilatori_e_interpreti.ipynb
-rw-r--r--   1 luigimalaguti  staff    20K Dec 10 17:48 2_Analisi_lessicale.ipynb
-rw-r--r--   1 luigimalaguti  staff   1.0K Dec  8 18:04 README.md
drwxr-xr-x@  5 luigimalaguti  staff   160B Dec 10 18:07 codes/
drwxr-xr-x@  7 luigimalaguti  staff   224B Dec 10 17:31 images/


In [1]:
cd codes/scanner

/Users/luigimalaguti/Desktop/Designs/LearnDynamicLanguages/2_Architettura/codes/scanner


In [2]:
ls -lah

total 16
drwxr-xr-x@ 4 luigimalaguti  staff   128B Dec 14 10:09 ./
drwxr-xr-x@ 5 luigimalaguti  staff   160B Dec 14 10:10 ../
-rw-r--r--  1 luigimalaguti  staff   1.5K Dec 14 10:09 scanner.py
-rw-r--r--  1 luigimalaguti  staff   358B Dec 14 10:08 wc.l


In [45]:
!flex wc.l

In [46]:
ls -lah

total 96
drwxr-xr-x@ 4 luigimalaguti  staff   128B Dec 10 18:10 ./
drwxr-xr-x@ 7 luigimalaguti  staff   224B Dec 10 18:10 ../
-rw-r--r--  1 luigimalaguti  staff    43K Dec 10 18:10 lex.yy.c
-rw-r--r--  1 luigimalaguti  staff   358B Dec 10 18:08 wc.l


In [47]:
!gcc lex.yy.c -o wc

In [48]:
ls -lah

total 200
drwxr-xr-x@ 5 luigimalaguti  staff   160B Dec 10 18:11 ./
drwxr-xr-x@ 7 luigimalaguti  staff   224B Dec 10 18:11 ../
-rw-r--r--  1 luigimalaguti  staff    43K Dec 10 18:10 lex.yy.c
-rwxr-xr-x  1 luigimalaguti  staff    51K Dec 10 18:10 wc*
-rw-r--r--  1 luigimalaguti  staff   358B Dec 10 18:08 wc.l


In [4]:
!./wc < lex.yy.c

1758 6161 44173


In [6]:
!wc lex.yy.c

    1758    6161   44173 lex.yy.c


Ricordiamoci che IPython ha la possibilità di interpretare anche i comandi bash e quindi di compilare ed eseguire il nostro programma.

Vediamo che come prima cosa abbiamo utilizzato il comanso ```flex```. Questo comando è il generatore di scanner. Passiamo ad esso il nostro codice per il comando wc e il generatore ci ritornerà un programma in grado di riconoscere le espressioni regolari specificate in esso.

Flex restituisce un programma C come abbiamo già detto. Infatti se utilizziamo il comando ```gcc``` per la compilazione di esso, otterremo un eseguibile in grado di contare le linee, parole e caratteri di un file in input.

Per passare un file al nostro comando wc abbiamo bisogno della ridirezione di input, il carattere ```<```.

Infine c'è un confronto tra l'esecuzione del nostro comando wc con il comando wc implementato di sistema in UNIX.

## Generazione mediante ply.lex

Come detto precedentemente, approfondiamo il generatore di scanner ply.

Ply è un modulo python che ci permette di descrivere come vogliamo che il generatore ci creii il nostro scanner.

Ovviamente la costruzione del nostro scanner è analoga alla costruzione appena vista con Lex in C, ma cambiano i metodi con coi comunichiamo al generatore.

In [4]:
from ply import lex

La prima cosa che dobbiamo indicare al generatore sono tutti i token.

La specifica dei token avviene in due modi differenti. Il primo consiste nella specifica dei token riservati, come ad esempio le istruzioni if, for, e altre del linguaggio python. I token riservati vengono specificati come un dizionario chiavi valori, dove la chiave corrisponde all'espressione regolare da associare al valore, ovvero il token.

In [5]:
reserverd = {
    'if': 'IF',
    'else': 'ELSE',
    'elif': 'ELIF',
    'while': 'WHILE',
    'def': 'DEF',
    'return': 'RETURN',
}

Il secondo modo per specificare i token consiste in una lista contenenti tutti i token che ci interessano

In [7]:
tokens = [
    'ID', 'NUMBER', 'COLON',
    'COMMA', 'PLUS', 'MINUS', 
    'MUL', 'DIV', 'AND',
    'OR', 'NOT', 'LPAR',
    'RPAR', 'SEP', 'EQ',
    'NEQ', 'LT', 'GT',
    'LE', 'GE', 'ASSIGN',
    'PRINT', 'PI'
] + list(reserverd.values())

e successivamente andiamo a specificare quali token rispondono a quali espressioni regolari. Ogni token può essere chiamato in python con la seguente scrittura ```t_TOKEN```. Questo ci permette di assegnare ad ogni token la rispettiva espressione regolare.

In [8]:
t_COLON = r':'
t_COMMA = r','
t_PLUS  = r'\+'
t_MINUS = r'-'
t_MUL   = r'\*'
t_DIV   = r'/'
t_LPAR  = r'\('
t_RPAR  = r'\)'
t_ASSIGN = r'='
t_SEP = r';'
t_LT = r'<'
t_GT = r'>'
t_LE = r'<='
t_GE = r'>='
t_EQ = r'=='
t_NEQ = r'!='
t_AND = r'and\b'
t_OR = r'or\b'
t_NOT = r'not\b'
t_PI = r'pi\b'
t_PRINT = r'print\b'

Possono esserci però alcuni token nella nostra lista che hanno bisogno di eseguire determinati compiti dopo il riconoscimento. Per definire le espressioni regolari di questi token e le relative azioni, bisogna definire funzioni con la stessa convensione utilizzata dalle variabili, ovvero essere chiamate con il nome del token preceduto da t_.

In [11]:
def t_ID(t):
    r'[a-zA-Z_][a-zA-Z_0-9]*'
    t.type = reserved.get(t.value,'ID')
    return t

def t_NUMBER(t):
    r'\d*\.\d+|\d+'
    v = float(t.value)
    if v == int(v):
        t.value = int(v)    
    else:
        t.value = v
    return t

Notiamo come l'espressione regolare nelle funzioni viene passata grazie a quella che è la docstring.

Inoltre passiamo al generatore informazioni su token speciali.

In [12]:
def t_newline(t):
    r'\n+'
    t.lexer.lineno += len(t.value)

t_ignore  = ' \t'

def t_error(t):
     print("Illegal character '%s'" % t.value[0])
     t.lexer.skip(1)

Abbiamo completato il nostro scanner. Quello che ci rimane da fare è creare una variabile ```lexer = lex.lex()``` per poi passargli dell'input da analizzare.

Il modulo ply da problemi di esecuzione proprio di quest'ultimo comando, quindi dal notebook corrente non riusciamo a vederlo, ma nella cartella ```codes``` è presente il file completo con anche il main di esecuzione. 

Per eseguire lo scanner di esempio basta digitare da terminale

```python
python scanner.py
```

Attenzione, bisogna prima scaricare il modulo ply con pip.